# Basics

Calculations of returns, as well as average returns and volatility of portfolios.

## Load Packages and Extra Functions

The notebook uses the functions `printmat()` and `printlnPs()` for formatted printing of matrices and numbers.

In [1]:
using Printf

include("jlFiles/printmat.jl")

printyellow (generic function with 1 method)

# Return Calculations

The return of holding the asset between $t-1$ and $t$ is

$
R_t = (P_t+D_t)/P_{t-1} - 1
$,

where $P_t$ is the price (measured after dividends) and $D_t$ is the dividend.

We can calculate the returns by a loop or by a more compact notation, see below.

In [2]:
P = [100;108;109]                     #price series, after dividends
D = [0;2;0]                           #dividends

R = zeros(length(P))                  #where to store the results
for t = 2:length(P)                   #P[2] is the 2nd element of P  
    R[t] = (P[t] + D[t])/P[t-1] - 1
end
R = R[2:end]
                                                #compact notation
R_alt = (P[2:end] + D[2:end])./P[1:end-1] .- 1  #notice the ./ and .-

println("    period    return    return (alt), %")
printmat([1:2 R*100 R_alt*100])

    period    return    return (alt), %
     1.000    10.000    10.000
     2.000     0.926     0.926



# Cumulating Returns

Net returns can be cumulated into a portfolio value as 

$
V_t = V_{t-1}(1+R_t)
$

where we need a starting value (initial investment) for the portfolio (a common choice is to normalise to $V_0=1$).

With log returns, $r_t=\log(1+R_t)$, we instead do 

$
\ln V_t = \ln V_{t-1} + r_t
$

If the return series is an excess return, add the riskfree rate to convert it to get net returns - and then cumulate as described above.

In [3]:
R   = [20;-35;25]/100                #returns for t=1,2,3
V   = cumprod(1 .+ R)                #V(t) = V(t-1)*(1+R(t)), starting at 1 in t=0
lnV = cumsum(log.(1 .+ R))           #lnV(t) = lnV(t-1) + r(t) 

println("   period    return       V        logV")
printmat([1:3 R V lnV])

println("Check that logV really equals log.(V). Also, try a loop instead")

   period    return       V        logV
     1.000     0.200     1.200     0.182
     2.000    -0.350     0.780    -0.248
     3.000     0.250     0.975    -0.025

Check that logV really equals log.(V). Also, try a loop instead


# Portfolio Return: Definition, Expected Value and Variance

With portfolio weights in the vector $w$, the portfolio return, the expected portfolio return and the portfolio variance can be computed as

$R_p = w'R$, 

$\text{E}R_p = w'\mu$ and 

$\text{Var}(R_p) = w'\Sigma w$

In [4]:
w = [0.8;0.2]
R = [10;5]/100          #returns of asset 1 and 2
Rp = w'R

println("portfolio weights (for the two assets): ")
printmat(w)
println("Returns of assets: ")
printmat(R)
printlnPs("Portfolio return: ",Rp)

portfolio weights (for the two assets): 
     0.800
     0.200

Returns of assets: 
     0.100
     0.050

Portfolio return:      0.090


In [5]:
μ = [9;6]/100                    #\mu and tab to get this
Σ = [256 96;
     96 144]/100^2

println("expected returns of the assets *100: ")
printmat(μ*100)

println("covariance matrix*100^2:")
printmat(Σ*100^2)

expected returns of the assets *100: 
     9.000
     6.000

covariance matrix*100^2:
   256.000    96.000
    96.000   144.000



In [6]:
ERp   = w'μ
VarRp = w'Σ*w

printlnPs("Expected portfolio return: ",ERp)
printlnPs("Portfolio variance and std:",VarRp,sqrt(VarRp))

Expected portfolio return:      0.084
Portfolio variance and std:     0.020     0.142


In [7]:
Σb = [256 -96;
      -96 144]/100^2

printlnPs("Portfolio std if the assets were negatively correlated: ",sqrt(w'Σb*w))

Portfolio std if the assets were negatively correlated:      0.118
